In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Package Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import autosklearn 
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport

In [ ]:
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

# Data Input

In [ ]:
df_train=pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
df_test=pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')

In [ ]:
df_train

# Data Analysis

In [ ]:
d1=ProfileReport(df_train.drop(['id'],axis=1))
d1.to_widgets()

# Train Test Split

In [ ]:
X=df_train.drop(['id','target'],axis=1)
y=df_train['target']

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)

# Model Building

## Linear Regression

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
pred1=lr.predict(x_val)

print(f"RMSE = {mean_squared_error(y_val,pred1,squared=False)}")
print(f"MAE = {mean_absolute_error(y_val,pred1)}")
print(f"R2_Score = {r2_score(y_val,pred1)}")

plt.figure(figsize=(10,10))
plt.scatter(x=y_val,y=pred1)
plt.show()

# SGDRegressor

In [ ]:
lr=SGDRegressor()
lr.fit(x_train,y_train)
pred2=lr.predict(x_val)

print(f"RMSE = {mean_squared_error(y_val,pred2,squared=False)}")
print(f"MAE = {mean_absolute_error(y_val,pred2)}")
print(f"R2_Score = {r2_score(y_val,pred2)}")

plt.figure(figsize=(10,10))
plt.scatter(x=y_val,y=pred2)
plt.show()

# Random Forest Regressor

In [ ]:
lr=RandomForestRegressor(n_jobs=-1,verbose=1,n_estimators=500)
lr.fit(x_train,y_train)
pred3=lr.predict(x_val)

print(f"RMSE = {mean_squared_error(y_val,pred3,squared=False)}")
print(f"MAE = {mean_absolute_error(y_val,pred3)}")
print(f"R2_Score = {r2_score(y_val,pred3)}")

plt.figure(figsize=(10,10))
plt.scatter(x=y_val,y=pred3)
plt.show()

# Bagging Regressor

In [ ]:
lr=BaggingRegressor(LinearRegression())
lr.fit(x_train,y_train)
pred3=lr.predict(x_val)

print(f"RMSE = {mean_squared_error(y_val,pred3,squared=False)}")
print(f"MAE = {mean_absolute_error(y_val,pred3)}")
print(f"R2_Score = {r2_score(y_val,pred3)}")

plt.figure(figsize=(10,10))
plt.scatter(x=y_val,y=pred3)
plt.show()

# Tensorflow Model

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Input,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

e=EarlyStopping(verbose=1,patience=5,restore_best_weights=True)
n=2

In [ ]:
model=Sequential()

model.add(Input(shape=(x_train.shape[1],)))

model.add(Dense(x_train.shape[1],activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(n*x_train.shape[1],activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n*x_train.shape[1],activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(x_train.shape[1],activation='relu'))

model.add(Dense(1,activation='relu'))

model.compile(metrics=[tf.keras.metrics.RootMeanSquaredError()],loss='mse',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
results=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=100,callbacks=[e])

In [ ]:
pred3=model.predict(x_test)

print(f"RMSE = {mean_squared_error(y_test,pred3,squared=False)}")
print(f"MAE = {mean_absolute_error(y_test,pred3)}")
print(f"R2_Score = {r2_score(y_test,pred3)}")

plt.figure(figsize=(10,10))
plt.scatter(x=y_test,y=pred3)
plt.show()

# Tensorflow model analysis

In [ ]:
d1=pd.DataFrame(results.history)
d1.iloc[:,[1,3]].plot(figsize=(10,10))

In [ ]:
d1=pd.DataFrame(results.history)
d1.iloc[:,[0,2]].plot(figsize=(10,10))